In [8]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D,Dropout
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
from keras import optimizers
from keras.optimizers import Adam
from keras.layers import BatchNormalization

Using TensorFlow backend.


In [0]:
input_directory = r"chest_xray/"

training_dir = input_directory + r"train"
validation_dir = input_directory + r"val"
testing_dir = input_directory + r"test"

In [0]:
def dir_file_count(directory):
    return sum([len(files) for r, d, files in os.walk(directory)])

In [11]:
rescale = 1./255
target_size = (150, 150)
batch_size = 163
class_mode = "categorical"


train_datagen = ImageDataGenerator(
    rescale=rescale,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)


train_generator = train_datagen.flow_from_directory(
    training_dir,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    shuffle=True)


validation_datagen = ImageDataGenerator(rescale=rescale)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=dir_file_count(validation_dir),
    shuffle = False)


Found 5216 images belonging to 2 classes.
Found 320 images belonging to 2 classes.


In [0]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

In [0]:
for layer in base_model.layers:
  layer.trainable = False

In [0]:
x = base_model.output
x = Dropout(0.5)(x)
x = GlobalAveragePooling2D(name='avg_pool')(x)
x = BatchNormalization()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation='softmax')(x) 
model = Model(inputs=base_model.input, outputs=predictions)

In [34]:
optimizer=optimizers.Adam()
loss='categorical_crossentropy'
metrics=['accuracy']

model.compile(optimizer, loss=loss, metrics=metrics)

EPOCHS = 5
BATCH_SIZE = 32
VALIDATION_STEPS=64
STEPS_PER_EPOCH=32
model.fit_generator(
    train_generator,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=validation_generator,
    validation_steps=VALIDATION_STEPS
   )

Epoch 1/5
32/32 [==============================] - 261s 8s/step - loss: 0.5350 - acc: 0.8459 - val_loss: 1.8589 - val_acc: 0.7375
Epoch 2/5
32/32 [==============================] - 242s 8s/step - loss: 0.3372 - acc: 0.8923 - val_loss: 4.2398 - val_acc: 0.6594
Epoch 3/5
32/32 [==============================] - 240s 8s/step - loss: 0.3312 - acc: 0.8903 - val_loss: 2.3144 - val_acc: 0.7219
Epoch 4/5
32/32 [==============================] - 244s 8s/step - loss: 0.2725 - acc: 0.9022 - val_loss: 2.5278 - val_acc: 0.6938
Epoch 5/5
32/32 [==============================] - 243s 8s/step - loss: 0.2644 - acc: 0.8974 - val_loss: 1.4405 - val_acc: 0.7031


In [56]:
img1 = image.load_img(r'chest_xray/test/NORMAL/NORMAL2-IM-0120-0001.jpeg', target_size=(150,150))
x1 = image.img_to_array(img1)
x1 = preprocess_input(x1)
x1 = np.expand_dims(x1, axis=0)
pred1 = model.predict(x1)[0]
prediction1=np.argmax(pred1)
prediction1

0

In [57]:
img2 = image.load_img(r'chest_xray/test/PNEUMONIA/person120_bacteria_571.jpeg', target_size=(150,150))
x2 = image.img_to_array(img2)
x2 = preprocess_input(x2)
x2 = np.expand_dims(x2, axis=0)
pred2 = model.predict(x2)[0]
prediction2=np.argmax(pred2)
prediction2

1